Cell 1: Import Required Libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import cv2

Cell 2: Define the Dataset Path and Emotion Labels

In [2]:
# Define the path to the dataset
dataset_path = r'C:\Users\sagar\OneDrive\Desktop\DS & ML Project\fer2013_dataset'

# Define the emotion labels
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# List contents of the dataset directory
print("Contents of dataset directory:")
print(os.listdir(dataset_path))


Contents of dataset directory:
['test', 'train']


Cell 2: Define Functions to Load and Preprocess Data

In [6]:
# Cell 2: Define functions to load and preprocess data

def load_data(dataset_path, emotion_labels):
    images = []
    labels = []
    
    for label in emotion_labels:
        emotion_dir = os.path.join(dataset_path, label)
        for img_name in os.listdir(emotion_dir):
            img_path = os.path.join(emotion_dir, img_name)
            img = tf.keras.preprocessing.image.load_img(img_path, color_mode='grayscale', target_size=(48, 48))
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            images.append(img_array)
            labels.append(emotion_labels.index(label))
    
    images = np.array(images)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=len(emotion_labels))
    
    return images, labels

# Define emotion labels and dataset path
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
dataset_path = 'C:\\Users\\sagar\\OneDrive\\Desktop\\DS & ML Project\\fer2013_dataset'

# Load data
faces, emotions = load_data(dataset_path, emotion_labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.2, random_state=42)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\sagar\\OneDrive\\Desktop\\DS & ML Project\\fer2013_dataset\\angry'

Cell 4: Define the Residual Block

In [4]:
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    return x

Cell 5: Build the Model

In [5]:
def build_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2))(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)
    
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, x)
    return model

input_shape = (48, 48, 1)
num_classes = emotions.shape[1]
model = build_model(input_shape, num_classes)

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


ValueError: Inputs have incompatible shapes. Received shapes (12, 12, 64) and (6, 6, 128)